## CLASS 3 AGENDA
* **Address questions/concerns from class 2**
* **Introducing the pandas library**
* **Working with the movielens movie ratings dataset**
* **Combining datasets (next notebook)**
* **Homework: NYC Taxi Data (2 notebooks)**

### Interesting use of pandas:

https://github.com/datadesk/kobe-every-shot-ever/blob/master/kobe-every-shot-ever.ipynb

### Repo for pandas practice:

https://github.com/guipsamora/pandas_exercises

As is almost always the case when working with **Python**, we are going to need more than just its basic functionality available to us as we develop our analytical pipelines. 

In order to have this additional functionality available (being able to use **pandas**), we will rely on a  couple `import` statements.

Here they are:

In [1]:
import pandas as pd
import numpy as np
from __future__ import print_function, unicode_literals, division

The code above did two things:

* Loaded in all of the functionality that **pandas** provides (`import pandas as pd`)
* Loaded in some additional functionality from a different package that **pandas** relies on called **NumPy** (`import numpy as np`)

Importantly, `pd` is now the alias (new name) for the entire `pandas` library and `np` is the alias for the `numpy` library. Instead of having to type `pandas.something` or `numpy.something` to access a given function, you can now just type `pd` or `np`. 

So what exactly is [**pandas**](http://pandas.pydata.org) and why the funny name (we will talk about [**NumPy**](http://www.numpy.org) a bit later)?

**pandas** is a Data Analysis Library written in and for the **Python** programming language and is a very loose acronym for **P**ython **An**alysis of **Da**taset**s** (or something like that anyway). 

It provides open source, easy-to-use data structures and data analysis tools and we will be relying on it heavily for the remainder of the course. Let's check the versions of `numpy` and `pandas` we are running:

In [2]:
print("Pandas version:",pd.__version__)
print("Numpy version:",np.__version__)

Pandas version: 0.18.1
Numpy version: 1.11.1


So, lets begin by loading in our first dataset, the Movielens 1M dataset.

This is a file that contains ~1,000,000 movie ratings taken from a website where users can rate a variety of movies on a 1-5 rating scale. Each rating is comprised of the following information:

* A user id, represented by a number (an `int`) 
* A movie id, also an `int` 
* A rating, an `int` that should vary from 1-5 (inclusive) 
* A timestamp, recording when the rating was made in seconds since UNIX epoch time (this is standardly set to 12:00AM January 1, 1970), also an `int`

The records in this file are arranged in the following order and format:

`UserID::MovieID::Rating::Timestamp`

We will use the `read_csv` function in pandas and try to simply load the dataset **as is** into a variable (actually an object) called `ratingData` and see if that gets it into a usable format (it won't).

In [3]:
'''GroupLens Research provides a number of collections of movie ratings data collected from users of MovieLens in the
late 1990s and early 2000s. The data provide ratings, movie metadata (genres and year), & demographic data about the 
users (age, zip code, gender, and occupation). Such data is often of interest in the development of recommendation
systems base on machine learning algorithms. The MovieLens 1M data set contains 1 million ratings collected from 6000
users on 4000 movies. It's spread across 3 tables: ratings, user info., and movie info. 
'''
# The read_csv function points to where the file is.
ratingData = pd.read_csv("../data/movieData/ratings.dat")
# It takes maybe about 0.5 sec. to load in and read the data

In [4]:
ratingData = pd.read_csv("../data/movieData/ratings.dat", sep='::')
# Note that the 'radings.dat' file does not have a header, so we don't know what the columns are.

/Users/markyashar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


If you look at the documentation for `read_csv`, you'll see that it is very large and provides for lots of different functionality. 

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

As a first pass, we just passed the path to the file in as a ```string``` to the `read_csv` function, without any other arguments. 

Lets take a look at the first few rows and see what we get using the `head` function on our newly loaded dataset `ratingData`. 

The `head` function returns the first 5 rows by default of the DataFrame you call it on. You can change it to be a larger or smaller number by passing in a positive `integer` into `head` as an argument like so: `ratingData.head(100)`.

The function `tail` does the exact same thing, except with the last records in the dataset.

In [5]:
''' Note that the 'DataFrame' is the basic data structure for Pandas to store rows & columns; it's a wrapper on top 
of the numpy array. Also, it's basically equivalent to an Excel spreadsheet, and it's C underneath the hood. It can
also be understood as a collection of database rows with each column stored as a numpy array, and where each column
is a single type...
'''
ratingData.head()   

,1,1193,5,978300760
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1197,3,978302268


In [6]:
ratingData.head(10)

,1,1193,5,978300760
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1197,3,978302268
5,1,1287,5,978302039
6,1,2804,5,978300719
7,1,594,4,978302268
8,1,919,4,978301368
9,1,595,5,978824268


Ok, well that looks terrible.
Lets diagnose the problems we see and make it unterrible: 

1. Everything is in a single column (so we can't separately look at ratings, user ids,movie ids, or timestamps)!
2. The first row is used as the name of the only column (we call this the **header**), which is no good, as the first record shouldn't be the header, but an actual record.
3. The timestamp is in a format that doesn't really tell us anything useful about when the ratings occurred.

So, lets use some of the additional functionality of `read_csv` to load this dataset in cleanly, and hopefully that will solve problems **1 and 2**.

In [7]:
'''Recall that the records in this file are arranged in the following order and format:
UserID::MovieID::Rating::Timestamp   -- so, we'd like to have 4 different (header) columns.
'''
ratingData = pd.read_csv("../data/movieData/ratings.dat",sep = "::",
                         names = ['UserID','MovieID','Rating','Timestamp'])
''' ADDITIONAL NOTES:
* sep : str, default ‘,’ -- Delimiter to use. If sep is None, will try to automatically determine this. Separators 
longer than 1 character and different from ‘s+’ will be interpreted as regular expressions, will force use of the 
python parsing engine and will ignore quotes in the data. Regex example: ‘rt’

* names : array-like, default None -- List of column names to use. If file contains no header row, then you should 
explicitly pass header=None

/Users/markyashar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


Don't worry about the `ParserWarning:` message (if you get one) as it doesn't affect what we are doing, and lets just take a look at the data now. I'll explain exactly what I did below.

In [40]:
ratingData.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


MUCH BETTER!

So, what did we just do?

The function `read_csv` has lots of functionality, as I had mentioned (and as you saw when you pulled up its documentation). One of its options is called `sep`, and allows you to provide your own separator for dividing the columns that you have in your dataset. 

The default separator for `read_csv` is the comma (`,`), since `csv` stands for **c**omma **s**eparated **v**alues. 

However, since `::` separated the fields in this dataset, we supplied that as an argument (again, as a `string`) to the argument `sep` instead.

A separate argument, `names`, allows you to pass in your own list of names, again as `strings`, to `read_csv` to be treated as the column names (or the **header**) of the dataset. Since we knew what the names for the columns should be, we put them in.

Now that our dataset looks more reasonable, lets do a couple brief sanity checks and then fix issue **3.**

Lets do a sanity check and make sure that:

1. All our data is in the format that we expect (everything is an `int`).
2. The ratings range across the values we expect (1-5 and nothing else).

The property `dtypes` is accessible from our `ratingData` object, and tells us the types of the data in all of our columns (which addresses **1.**).

In [8]:
ratingData.dtypes

UserID       int64
MovieID      int64
Rating       int64
Timestamp    int64
dtype: object

Ok, so everything appears to be an `int` as `int64` is an `integer` (positive or negative whole number) data type that can represent very large numbers.

As an important aside about **pandas**, all the values in a given column have to be of the same type. So, **if even one value was not a whole number, (1.0 for example), the values for the entire column would be inferred to be something else (either a `float64` or an `object` if any of the entries were `strings`).**

Now let's address **2.** by looking at all of the unique entries in the `Rating` column using `unique` (we expect there to be 5 unique values, 1-5 inclusive).

As long as your column names do not contain strange characters (spaces and escape characters like !\/), you can simply access the values in a column by doing `dataFrameName.columnName` where `dataFrameName` is the name of your `DataFrame` object and `columnName` is the name of your column.

The function `unique` is accessible from our `ratingData` object, and simply returns all of the unique values within our column of choice as a `List`. You can also use `nunique` to simply return the number of distinct elements (as opposed to the elements themselves:

In [9]:
print("Unique values:",ratingData.Rating.unique())
print("Number of unique values:",ratingData.Rating.nunique())

Unique values: [5 3 4 2 1]
Number of unique values: 5


In [10]:
# Try calling the .unique() function by using other column names in the dataset.
# Data science is all about getting to know your data, so get to know the unique values of the columns in the dataset!
#YOUR CODE HERE
print("Unique values:",ratingData.UserID.unique())
print("Number of unique values:",ratingData.UserID.nunique())
print("Unique values:",ratingData.MovieID.unique())
print("Number of unique values:",ratingData.MovieID.nunique())
print("Unique values:",ratingData.Timestamp.unique())
print("Number of unique values:",ratingData.Timestamp.nunique())
print("Unique values:",ratingData.Rating.unique())
print("Number of unique values:",ratingData.Rating.nunique())

Unique values: [   1    2    3 ..., 6038 6039 6040]
Number of unique values: 6040
Unique values: [1193  661  914 ..., 2845 3607 2909]
Number of unique values: 3706
Unique values: [978300760 978302109 978301968 ..., 956704219 964828799 956704746]
Number of unique values: 458455
Unique values: [5 3 4 2 1]
Number of unique values: 5


However, if your dataset contains weird column names, you have another way of accessing columns:

In [11]:
ratingData["Rating"].unique()

array([5, 3, 4, 2, 1])

In this way of accessing the `Rating` column, we have to pass the name of the column as a `string` (in quotes "").

What if we wanted to access multiple columns? Here's how you would do that (this is the only way):

In [12]:
ratingData[["Rating","UserID"]].head()
# Is equivalent to:
multipleColumns = ["Rating","UserID"]
ratingData[multipleColumns].head()

,Rating,UserID
0,5,1
1,3,1
2,3,1
3,4,1
4,5,1


In the multi-column case, you must pass the columns you are interested in as a `List` of `string` values (or as a `variable` that points to that list).

Now, on to fixing issue **3**.

**pandas** has pretty fantastic date conversion functionality, as long as you know the format of the date data you are using. We know the format of our `Timestamp` column, so we are good to go.

As a refresher, it was seconds since epoch time (12:00AM January 1, 1970).

The pandas library has a function called `to_datetime` that, when given a column, and some optional parameters, converts the timestamp into nice, prettily formatted text.

Lets create a new column called `FormattedTimestamp` by formatting our `Timestamp` column:

In [13]:
'''pandas is generally oriented towards working with arrays of dates, whether used as an axis index or a column
in a DataFrame. The 'to_datetime' method parses many different kinds of date representations. Standard date formats
like ISO8601 can be parsed very quickly (for example). We want to transform the timestamp to a date. Note that leap-
year issues and calculations have been taken care of in the software/library/module
'''
ratingData['FormattedTimestamp'] = pd.to_datetime(ratingData.Timestamp,unit = 's')
ratingData.head(5)

,UserID,MovieID,Rating,Timestamp,FormattedTimestamp
0,1,1193,5,978300760,2000-12-31 22:12:40
1,1,661,3,978302109,2000-12-31 22:35:09
2,1,914,3,978301968,2000-12-31 22:32:48
3,1,3408,4,978300275,2000-12-31 22:04:35
4,1,2355,5,978824291,2001-01-06 23:38:11


Ok, there is a lot going on here, we are creating a new column `FormattedTimestamp` from a computation on another column, so lets work through it. 

To create a new column, you just use the same syntax as when you want to select a column, except you use a name that isn't found in the dataset's column list. 

Everything following the `=` sign in the expression is what you want to put into that new column.

And what we did following the equals sign is:

1. We passed the `Timestamp` column of our ratingData dataset as a mandatory parameter.
2. We supplied an optional string parameter called `unit` with the unit of our data as a `string`.

(Look at the documentation, and you can see there is lots more stuff you can do with `to_datetime`!).

Now, we can simply remove the old `Timestamp` column, since we don't need it anymore!

To do so, you simply pass the name of the dataframe and the column you are trying to delete to the `del` function:

In [14]:
print("Columns before removal: ", ratingData.columns)
del ratingData["Timestamp"]
print("Columns after removal: ", ratingData.columns)

Columns before removal:  Index([u'UserID', u'MovieID', u'Rating', u'Timestamp', u'FormattedTimestamp'], dtype='object')
Columns after removal:  Index([u'UserID', u'MovieID', u'Rating', u'FormattedTimestamp'], dtype='object')


The `del` operation simply deletes the columns you specify from the given `DataFrame`. Remember that once you've deleted a given column, you can't delete it again, so executing the above cell muliple times will throw an error.

One more thing about dates before we move on, once you've got them converted to the pretty format we saw above, you can access all kinds of information from each date by calling the `dt` module from within the column that stores your dates.

So, as an example, getting the year of every row in our dataset is as simple as:

In [15]:
ratingData.FormattedTimestamp.dt.dayofyear

0          366
1          366
2          366
3          366
4            6
5          366
6          366
7          366
8          366
9          366
10           6
11         366
12         366
13         366
14         366
15         366
16           6
17         366
18         366
19         366
20         366
21         366
22         366
23           6
24         366
25           6
26         366
27         366
28           6
29           6
          ... 
1000179    116
1000180    117
1000181    116
1000182    128
1000183    166
1000184    117
1000185    117
1000186    116
1000187    117
1000188    210
1000189    128
1000190    117
1000191    210
1000192    116
1000193    117
1000194    210
1000195    117
1000196    116
1000197    117
1000198    117
1000199    117
1000200    116
1000201    128
1000202    116
1000203    117
1000204    117
1000205    116
1000206    116
1000207    117
1000208    117
Name: FormattedTimestamp, dtype: int64

In [16]:
ratingData.FormattedTimestamp.dt.hour

0          22
1          22
2          22
3          22
4          23
5          22
6          22
7          22
8          22
9          22
10         23
11         22
12         22
13         22
14         22
15         22
16         23
17         22
18         22
19         22
20         22
21         22
22         22
23         23
24         22
25         23
26         22
27         22
28         23
29         23
           ..
1000179    23
1000180     2
1000181    23
1000182    16
1000183     8
1000184     2
1000185     2
1000186    23
1000187     2
1000188    23
1000189    16
1000190     2
1000191    23
1000192    23
1000193     2
1000194    23
1000195     2
1000196    23
1000197     2
1000198     2
1000199     2
1000200    23
1000201    16
1000202    23
1000203     2
1000204     2
1000205    23
1000206    23
1000207     2
1000208     2
Name: FormattedTimestamp, dtype: int64

Other properties (like day, hour, day of month, etc.) are available as well.

Again, just check the [timestamp documentation.](http://pandas.pydata.org/pandas-docs/stable/timeseries.html).

In [23]:
#try using the other functions found in the dt module: 
#call ratingData.FormattedTimestamp.dt and the use the Tab key
#to see what else is offered
    
#YOUR CODE HERE
print(ratingData.FormattedTimestamp.dt.second)
print(ratingData.FormattedTimestamp.dt.minute)
print(ratingData.FormattedTimestamp.dt.weekofyear)
print(ratingData.FormattedTimestamp.dt.weekday)
print(ratingData.FormattedTimestamp.dt.year)


0          40
1           9
2          48
3          35
4          11
5          48
6          59
7          59
8          48
9          48
10         48
11         12
12          1
13         24
14         13
15         28
16         48
17         37
18         33
19         59
20         45
21         40
22         55
23         35
24         43
25         11
26         33
27         55
28         39
29         48
           ..
1000179    24
1000180    35
1000181    52
1000182    18
1000183    14
1000184    29
1000185    58
1000186    19
1000187    34
1000188    54
1000189    35
1000190    33
1000191    42
1000192    17
1000193    48
1000194    59
1000195    48
1000196     6
1000197    48
1000198    54
1000199     7
1000200    19
1000201    22
1000202    16
1000203    38
1000204    41
1000205    27
1000206     6
1000207    48
1000208    29
Name: FormattedTimestamp, dtype: int64
0          12
1          35
2          32
3           4
4          38
5          37
6          33
7        

One note about the outputs of calling these datetime **properties.** When you call the property on a single value (like a single row) or on an entire column you will get a `Series` object returned to you, which is the **pandas** representation of a single column.

`Series` objects are different from `DataFrame` objects (which we've been working with exclusively thus far) because they can be appended (attached) as new columns to your dataset (which is already a `DataFrame` object) without any problems.

So if you want to store the `year` of every row in a new column called `year` then do:

In [24]:
ratingData["year"] = ratingData.FormattedTimestamp.dt.year

In [26]:
#make another column called "month" and call the appropriate function to store the month of each rating
#do the same thing for "day", just to get a good handle on the kinds of things you can do
#YOUR CODE HERE

ratingData["month"] = ratingData.FormattedTimestamp.dt.month
ratingData["day"] = ratingData.FormattedTimestamp.dt.day
ratingData.head(10)

,UserID,MovieID,Rating,FormattedTimestamp,year,month,day
0,1,1193,5,2000-12-31 22:12:40,2000,12,31
1,1,661,3,2000-12-31 22:35:09,2000,12,31
2,1,914,3,2000-12-31 22:32:48,2000,12,31
3,1,3408,4,2000-12-31 22:04:35,2000,12,31
4,1,2355,5,2001-01-06 23:38:11,2001,1,6
5,1,1197,3,2000-12-31 22:37:48,2000,12,31
6,1,1287,5,2000-12-31 22:33:59,2000,12,31
7,1,2804,5,2000-12-31 22:11:59,2000,12,31
8,1,594,4,2000-12-31 22:37:48,2000,12,31
9,1,919,4,2000-12-31 22:22:48,2000,12,31


Let's move on and get a better feel for our dataset now.

What if we wanted to know the exact shape of our dataset? That is, the *exact* number of rows and columns found in it? (I told you there were ~1,000,000 ratings here, but exactly how many are there?

We can use another **property** that is available in our `ratingData` object called `shape`:

In [27]:
# Every array has a 'shape', a typle indicating the size of each dimension. 'shape' gives number of rows & columns
(numRows,numColumns) = ratingData.shape

In [28]:
print(numRows)
print(numColumns)

1000209
7


In [29]:
# The 'describe()' method produces multiple summary statistics in one shot. It gives us statistics on numeric
# columns
ratingData.describe()

,UserID,MovieID,Rating,year,month,day
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,2.000126e+03,8.710371e+00,1.544069e+01
std,1.728413e+03,1.096041e+03,1.117102e+00,4.223923e-01,2.717470e+00,8.888445e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+03,1.000000e+00,1.000000e+00
25%,1.506000e+03,1.030000e+03,3.000000e+00,2.000000e+03,7.000000e+00,7.000000e+00
50%,3.070000e+03,1.835000e+03,4.000000e+00,2.000000e+03,9.000000e+00,1.700000e+01
75%,4.476000e+03,2.770000e+03,4.000000e+00,2.000000e+03,1.100000e+01,2.200000e+01
max,6.040000e+03,3.952000e+03,5.000000e+00,2.003000e+03,1.200000e+01,3.100000e+01


In [30]:
ratingData.info()   # info() gives us information such as how many values are non-null (where null = missing value)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 7 columns):
UserID                1000209 non-null int64
MovieID               1000209 non-null int64
Rating                1000209 non-null int64
FormattedTimestamp    1000209 non-null datetime64[ns]
year                  1000209 non-null int64
month                 1000209 non-null int64
day                   1000209 non-null int64
dtypes: datetime64[ns](1), int64(6)
memory usage: 53.4 MB


The `shape` property (which doesn't have any documentation, unfortunately), tells us the number of dimensions and the number of values in each dimension in our dataset (we can have 1, 2, or more dimensions in our dataset, after all), as a tuple (a very common datatype in **Python** that is of a fixed size and cannot be changed).

As a brief aside, you saw that I just said **property** and not **function** right? 

That means, this value is available to us as part of the dataset intrinsically and cannot be changed based on inputs (in CS parlance, it is read-only), and doesn't have to be called like `unique(), head(), tail(), max(), mean()` and all the other functions that are re-computed on the dataset because they can effectively change! 

(Another example property that is available to you and that you've already used is `dtypes`)

Ok, back to `shape`.

The number of dimensions in our dataset is the arity of the tuple (the number of commas + 1) and the numbers between commas tell us the number of values in that dimension. So the **arity is 2 because all we have are rows and columns.** 

So, we have **1,000,209 distinct values in the first dimension** (here they are our ratings, or **rows**) and **4 distinct values in the second dimension** (here the second dimension is our **columns**, and is as we expect, since we just added a new column and deleted an old one).

Now that you have a bit of **pandas** functionality at your disposal, you should give me the answer to the following questions by writing a bit of code:

1. How many users are there in the dataset?
2. How many movies are in the dataset?
3. How many unique times are in the dataset?

**Hint: All you need is the `unique()` function and the `shape` property to answer these questions!**

In [35]:
## WRITE YOUR CODE HERE, replace "pass" with your code in all 3 cases
## Write a print function to display your results.
print("Number of users in the dataset", ratingData.UserID.unique().shape)
print("Number of movies in the dataset", ratingData.MovieID.unique().shape)
print("Number of unique times in the dataset", ratingData.FormattedTimestamp.unique().shape)

Number of users in the dataset (6040,)
Number of movies in the dataset (3706,)
Number of unique times in the dataset (458455,)


Awesome! You've written your first bit of code using **pandas** and have actually answered some useful questions about this dataset! Pats on the back all around, and lets keep exploring!

Now, let's look at the ratings in the dataset to get a feel for their central values and spread:

In [36]:
print("The average rating in the dataset is:",ratingData.Rating.mean())
print("The middle rating in the dataset is:",ratingData.Rating.median())
print("The standard deviation of the ratings is:",ratingData.Rating.std())

The average rating in the dataset is: 3.58156445303
The middle rating in the dataset is: 4.0
The standard deviation of the ratings is: 1.11710184538


So it looks like the people in our dataset don't like to rate movies too low, as the movies have  an average rating >3 (which is supposed to be average on a 1-5 scale).

The functions `mean`,`median`, and `std` compute exactly what you expect (the mean, median, and standard deviation {or spread} of a given set of values) and can even be applied to the entire dataset (although in our case, that doesnt make too much sense as all the other columns, except `Timestamp` are numeric mappings of categorical data).

In [37]:
ratingData.mean()

UserID     3024.512348
MovieID    1865.539898
Rating        3.581564
year       2000.126168
month         8.710371
day          15.440691
dtype: float64

In [38]:
ratingData.median()

UserID     3070.0
MovieID    1835.0
Rating        4.0
year       2000.0
month         9.0
day          17.0
dtype: float64

In [214]:
ratingData.std()

UserID     1728.412695
MovieID    1096.040689
Rating        1.117102
year          0.422392
month         2.717470
day           8.888445
dtype: float64

In [39]:
ratingData.min()

UserID                                  1
MovieID                                 1
Rating                                  1
FormattedTimestamp    2000-04-25 23:05:32
year                                 2000
month                                   1
day                                     1
dtype: object

Other descriptive stats can be computed as well (min/max, sum, variance, skew, kurtosis, etc.). Just take a look at the [descriptive statistics](http://pandas.pydata.org/pandas-docs/stable/basics.html#descriptive-statistics) documentation!  

In [42]:
#Look at the descriptive statistics documentation and print a couple more descriptive stats
#on just the "Rating" column. You get to choose what stats you want to print
#YOUR CODE HERE
print("Rating mean: ", ratingData['Rating'].mean())
print("Rating maximum: ", ratingData['Rating'].max())
print("Rating mode: ", ratingData['Rating'].mode())
print("Rating minimum: ", ratingData['Rating'].min())
print("Rating median: ", ratingData['Rating'].median())
# ratingData_Rating=ratingData['Rating']
print("Rating median: ", ratingData.Rating.median())
print("Rating kurtosis: ", ratingData.Rating.kurtosis())

Rating mean:  3.58156445303
Rating maximum:  5
Rating mode:  0    4
dtype: int64
Rating minimum:  1
Rating median:  4.0
Rating median:  4.0
Rating kurtosis:  -0.35197080314


Now time for a little trick! 

If you want all these basic statistics and a few more, just use the `describe` function, which works just like the others mentioned above (you can call it on just a single column, or the whole dataset).

In [43]:
ratingData.describe()

,UserID,MovieID,Rating,year,month,day
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,2.000126e+03,8.710371e+00,1.544069e+01
std,1.728413e+03,1.096041e+03,1.117102e+00,4.223923e-01,2.717470e+00,8.888445e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+03,1.000000e+00,1.000000e+00
25%,1.506000e+03,1.030000e+03,3.000000e+00,2.000000e+03,7.000000e+00,7.000000e+00
50%,3.070000e+03,1.835000e+03,4.000000e+00,2.000000e+03,9.000000e+00,1.700000e+01
75%,4.476000e+03,2.770000e+03,4.000000e+00,2.000000e+03,1.100000e+01,2.200000e+01
max,6.040000e+03,3.952000e+03,5.000000e+00,2.003000e+03,1.200000e+01,3.100000e+01


In [80]:
ratingData.Rating.value_counts()   # value_counts() tells us how many of our values lie in the different ratings.

4    348971
3    261197
5    226310
2    107557
1     56174
Name: Rating, dtype: int64

Now lets learn how to subselect values within dataframes.

Subselection in **pandas** works along the following very common pattern:

1. You create a condition that can be evaluated to either **true** or **false** for every row in the dataset and store the outcome in a variable (this is traditionally called a **mask**).
2. You apply that **mask** onto your `DataFrame` (dataset).

Let's try this subselection + application with the ratings in our dataset by only getting all of the really low ratings (lets say low ratings are those that are < 3) in our dataset.

We will create a mask that expresses our "crappy ratings" condition, and then apply that mask to our dataset.

In [44]:
# Note that a mask eventually results in false values being thrown away, i.e., we're filtering stuff out.
crappyRatingMask = ratingData.Rating < 3
crappyRatings = ratingData[crappyRatingMask]
crappyRatings
# print(crappyRatingMask)

,UserID,MovieID,Rating,FormattedTimestamp,year,month,day
67,2,1213,2,2000-12-31 21:34:18,2000,12,31
73,2,434,2,2000-12-31 22:02:54,2000,12,31
75,2,3107,2,2000-12-31 22:00:02,2000,12,31
83,2,902,2,2000-12-31 21:41:45,2000,12,31
91,2,3256,2,2000-12-31 21:57:19,2000,12,31
114,2,3699,2,2000-12-31 21:46:13,2000,12,31
125,2,2427,2,2000-12-31 21:58:33,2000,12,31
133,2,95,2,2000-12-31 22:02:23,2000,12,31
148,2,21,1,2000-12-31 21:57:19,2000,12,31
151,2,1090,2,2000-12-31 21:36:20,2000,12,31


In [45]:
print(crappyRatings.Rating.unique())
# This is a sanity check to make sure that our filtered data contains only low ratings

[2 1]


Awesome, this worked as expected!
How many of these crappy ratings are there? (Replace **pass** in the code cell below with your answer)

In [46]:
# pass
print("Ratings value_counts(): " , crappyRatings.Rating.value_counts())
print("Ratings count: ", crappyRatings.Rating.count())

Ratings value_counts():  2    107557
1     56174
Name: Rating, dtype: int64
Ratings count:  163731


Now it is time for you to show how much you've learned. Give me the answers to the following questions:

1. What was the average rating in January?
* What was the average crappy rating in January?

**Hint:** Although there are lots of ways you can tackle these questions (many of which you will learn soon). I suggest you use the following procedure:

1. Create a new column in `ratingData` and call it "month", so that it tells you the month of every row
* Create a new dataset of:
  1. all January records and store it in `januaryRatings`
  * create a mask of crappy ratings using that dataset called `crappyJanuaryRatingsMask`
  * apply that `crappyJanuaryRatingsMask` to your dataset and store it in a new dataset called `crappyJanuaryRatings`
  * compute the mean of the `Rating` column for both `januaryRatings` and `crappyJanuaryRatings`

In [48]:
month = ratingData.month
# print(month)
ratingData[month==1]
crappyRatings[crappyRatings.month == 1].Rating.mean()  # this is the way instructor did it

1.698259705488621

In [62]:
# A slightly different and more thorough approach
month = ratingData.month
januaryRatings = ratingData[month==1]
print("The average rating in January was: ", januaryRatings.Rating.mean())
crappyJanuaryRatingsMask = januaryRatings.Rating < 3
crappyJanuaryRatings = januaryRatings[crappyJanuaryRatingsMask].Rating
print("The average crappy rating in January was: ", crappyJanuaryRatings.mean())

The average rating in January was:  3.54299583911
The average crappy rating in January was:  1.69825970549


Ok, now that you know how to do some basic subselection, sorting, and calculations on data, we are going to do something a bit more complicated, and start subdividing our data into groups to be able to answer some more general questions about our dataset.

Once you have this functionality down, you will be able to: 

1. Answer more interesting kinds of questions
* Answer the questions above using fewer lines of code 

Lets say you wanted to know or do the following:

1. **In what month did users rate the most movies?**
2. **What month had the highest average rating?**
3. **Remove users with too few ratings (lets say < 30) and reanswer these same questions**

Our approach here will be:

* Learn to use the `groupby` functionality of **pandas** to create subgroups of our ratings based on either the `month` the ratings were given or on the `UserID` of the rater.
* Apply an aggregating function to these groups to return:
  * The `size` of each group (since the `size` of each group is either the number of ratings in that `month` or the number of ratings for that `UserID`)
  * The `mean` of the `Rating` column within each group
  * A filtered version of the original dataset so that only the groups that are large enough (when grouping on `UserID`, those users that have made enough ratings) are returned.

The `groupby` function in **pandas** is analagous to the grouping operations you may be familiar with if you've ever used any **SQL** variants. 

A generic **SQL** translation of **1.**, for example, would look something like:

```
SELECT month,numRatings FROM (SELECT month,size(month) AS numRatings
FROM ratingData
GROUP BY month) WHERE numRatings = MAX(numRatings)
```

(If this looks like wizardry, don't worry, I'm just trying to show this to those users that are familiar with SQL and any of its variants; this is the only SQL you will see all weekend!)

Grouping can get very complicated, but as a first pass you can think of it as a way to split your dataset into non-overlapping subsets along any axis (along rows or columns, in our case). 

The values along which you **group** your dataset are traditionally called **keys**, so **each key should be unique to each group, and each group can have at most one key associated with it** (although the key for identifying each group can be really complicated).

Once you've grouped your dataset, the `GroupBy` object isn't too useful by itself. It becomes useful when you apply a **transformation** to it and get a new dataset back. 

We typically call this **transformation** an **aggregation**, as we are getting some aggregate value back for each group.

The **aggregation** functions we will be using for questions **1.,2.** are `size` and `mean`.

So, enough explaining, lets get to some hacking.

Lets address grouping in the context of answering our first question:

* **In what month did users rate the most movies?**

In **pandas**, to create groups, you must create a `GroupBy` object (more on what that is later) from your `DataFrame` object (your dataset) by passing the values along which you want to group to the `groupby` function.

We want to **groupby** the **month** column and store it:

In [63]:
# In this case, grouping and aggregating will take the rows for each month and group them together. Recall that the
# groupy functionality is common in SQL.
monthGroups = ratingData.groupby("month")
monthGroups

This object, called a `DataFrameGroupBy` is a rearranging of the rows in the dataset into distinct collections, called groups. Let's take a look at the groups:

In [67]:
print(monthGroups.describe())

                    UserID        MovieID         Rating           year  \
month                                                                     
1     count   23072.000000   23072.000000   23072.000000   23072.000000   
      mean     2003.050061    1982.618715       3.542996    2001.299931   
      std      1651.173209    1145.835810       1.075643       0.608711   
      min         1.000000       1.000000       1.000000    2001.000000   
      25%       531.000000    1079.000000       3.000000    2001.000000   
      50%      1601.000000    1997.000000       4.000000    2001.000000   
      75%      3001.750000    2985.000000       4.000000    2001.000000   
      max      5996.000000    3952.000000       5.000000    2003.000000   
2     count   12128.000000   12128.000000   12128.000000   12128.000000   
      mean     2533.465287    2033.421257       3.523664    2001.452507   
      std      1742.463938    1144.649503       1.089588       0.703198   
      min        19.00000

We can't print `monthGroups` itself, as its not a `DataFrame`, but a rearranged representation:

In [68]:
print(monthGroups)

And then we want to get the number of records (rows) in each group in our `monthGroups` object using the `size` function (which is accessible from this object):

In [69]:
# size() will tell us how many rowa are in each month (where we've grouped by month for this case).
ratingsPerMonth = monthGroups.size()
# meanratingsPerMonth = monthGroups.mean()
ratingsPerMonth #this will simply print the result inside of this notebook so we can see it
#to print to the screen outside of the notebook, you would have to say "print(ratingsPerMonth)"

month
1      23072
2      12128
3       8537
4      19407
5      74278
6      61110
7      97004
8     188674
9      56791
10     45500
11    295461
12    118247
dtype: int64

And just to make it nice and clean, we will simply output the month with the largest number of ratings using the `max` function:

In [70]:
monthWithMostRatings = ratingsPerMonth[ratingsPerMonth==ratingsPerMonth.max()]
#same as above, print to notebook without having to say "print highestRating"
monthWithMostRatings
# tells us that november had the most ratings

month
11    295461
dtype: int64

Got it? Good!

One more thing before you try it yourself. Once you've created a `groupby` object, you don't have to select everything in the object to perform an aggregation, and can subselect a given column within each group (**Hint, Hint!**)

Now you try answering question **2.**:

* **What was the average rating given to movies in each month of the year?**

Create the objects `avgMonthlyRatings` and `highestAvgRating`, that allow you to answer the question and print them to the screen:

In [72]:
# pass  -- What was the average rating given to movies in each month of the year?
avgMonthlyRatings = monthGroups.Rating.mean()   # remember, we group by months first, and that's were 'monthGroups' 
                                                # comes in
avgMonthlyRatings
# March had the lowest average monthly rating

month
1     3.542996
2     3.523664
3     3.456952
4     3.522028
5     3.601847
6     3.616528
7     3.617985
8     3.566315
9     3.602279
10    3.609143
11    3.571737
12    3.581435
Name: Rating, dtype: float64

In [73]:
# As a bit of a sanity check, I've calculated below the maximum rating given to moves in each month of the year
maxMonthlyRatings = monthGroups.Rating.max()
maxMonthlyRatings

month
1     5
2     5
3     5
4     5
5     5
6     5
7     5
8     5
9     5
10    5
11    5
12    5
Name: Rating, dtype: int64

These aggregations are already implemented in the `GroupBy` object and can be called directly from the object, but this is not generally the way this is handled.

Under the hood, **pandas** is passing the function we apply (`mean` or `size` in our case) to another function called `aggregate` and that function is actually doing the heavy lifting.

**The `aggregate` function operates on the entire group within the groupby, so any group-based operations must use an aggregating function.**

So, what is actually happening when we wrote our code to answer question **1.** above like this:

`ratingsPerMonth = monthGroups.size()`

Was actually being implemented more like this:

``ratingsPerMonth = monthGroups.aggregate(size)``

or this:

``ratingsPerMonth = monthGroups.agg(size)``

So, this means that whenever we want to do more complicated transformations or reductions of our data, on a per-group level, we should supply our function(s) of interest to `aggregate` or the shorhand `agg` function.

A couple more explanations before we tackle filtering for answering question **3.**

You can pass multiple functions to `aggregate` as a `List`, if you want multiple transformations applied to the data, like so:

In [75]:
# We might want to do multiple aggregations at once ...
monthGroups.Rating.agg([np.mean,np.size,np.std])  # 4 columns are output from this
monthGroups.Rating.agg([np.mean,np.size,np.std]).reset_index()  # 'reset_index' redoes the index and starts over

,month,mean,size,std
0,1,3.542996,23072,1.075643
1,2,3.523664,12128,1.089588
2,3,3.456952,8537,1.079460
3,4,3.522028,19407,1.096929
4,5,3.601847,74278,1.137355
5,6,3.616528,61110,1.110452
6,7,3.617985,97004,1.092208
7,8,3.566315,188674,1.122584
8,9,3.602279,56791,1.113047
9,10,3.609143,45500,1.087170


In all of these cases, you will notice that we have to call the functions using the **NumPy** module (but using the `np` alias), which was installed when you got **pandas** working on your system (since **NumPy** is a dependency of **pandas** and gives **pandas** all of the math and matrix wrangling functionality it relies on behind the scenes).

**NumPy** is a really powerful matrix and math library in **Python** and has lots of functionality we won't go into here, so if you're interested, head over to their [website](http://www.numpy.org) to learn more!

Now, lets move on to filtering.

Lets go through how to answer question **3.**:

* **Remove users with too few ratings (lets say < 30) and reanswer these same questions**

Here is our pipeline:

1. `groupby ratingData on "UserId"` and call this new `GroupBy` object `userGroups`
* `filter` so that only groups (users) containing > 30 ratings are kept in a new `DataFrame`, called `filteredRatings`
* `groupby filteredRatings on "month"` and call this new `GroupBy` object `filteredMonthGroups`
* recompute `mean` and `size` statistics on the `Rating` column in `filteredMonthGroups` and store it in a variable called `filteredMonthAggs`
* use `max` to see if the month when the largest rating mean and rating size have changed

In [77]:
# Here was use groupby ratingData on "UserId" and cal this new 'GroupBy' object 'userGroups'
userGroups = ratingData.groupby("UserID")

This first step is very similar to what we did before, except we are grouping on a different column, `UserID`.

Now comes the more challenging part, using `filter`, and involves learning a bit about **anonymous functions**. Here goes:

In [78]:
# Here, we want to filter so that only groups (users) containing > 30 ratings are kept in a new DataFrame, which
# we'll call 'filterdRatings'.
def enoughRatings(currGroup):
    return currGroup.Rating.size >= 30

In [79]:
# Here, we use anonymous/labmda functions
filteredRatings = userGroups.filter(lambda currGroup: currGroup.Rating.size >= 30)
filteredRatings2 = userGroups.filter(enoughRatings)  # This alternative approach indicates that it may be possible
                                                     # to do this without using lambda/anonymous functions

In [80]:
print(filteredRatings.shape)
print(filteredRatings2.shape)

(982040, 7)
(982040, 7)


**AAAAA WHAT IS THAT? lambda? x? What are these things?**

Ok, lets take a breath and work through this...

The `filter` function takes a function as an input and requires that the function you pass to it return either `True` or `False` on a per-group basis. 

It then returns a new `DataFrame` object, sorted into the groups you grouped on initially, with all of the groups removed that don't satisfy the constraints of the function you passed to it (that is, removing those groups that, when the function is applied to them, return `False`).

So, why are we using this weird `lambda` thing? 

Well, `lambda` is the keyword for creating an **anonymous function** in **Python**.

**anonymous functions** are functions that you define within some restricted place that:

1. Usually accomplish some very minimal functionality
2. Don't need the syntactic sugar that functions usually come with because of 1. and to maintain the compactness (and hopefully clarity) of your code.

To declare an anonymous function you:

1. type `lambda`, followed by
* arbitrary names for the parameters that function accepts (x,y, etc.) separated by commas, followed by
* a colon, which tells **Python** that we are done with specifying the parameters, followed by
* the expression that defines how the function operates on the parameters. This expression will dictate what the function returns.

Here's a really dumb anonymous function, stored in the variable `dumb`:

In [81]:
# Lambda functions are really just simple functions consisting of a single statement, the result of which is the 
# return value. They are defined using the 'lambda' keyword, which has no meaning other than "we are declaring an
# anonymous function."
dumb = lambda x: x + 1
print(dumb(6))

7


So:
`lambda x: x.Rating.size >= 30` means:

1. **This function accepts a single parameter (in our case the group) arbitrarily called x.**
* **It operates on this parameter, x, by finding some attribute in it called "Rating" (which it definitely has from earlier steps) and checking whether its size property is greater than or equal to 30.**
* **Because this function is checking a condition, it will return either `True` or `False` for every parameter you pass to it**

If you understand that, you grok **anonymous functions**.

**However, if you don't understand WTF just happened, you can actually write your own function, and just pass it directly to the filter!**

So, instead of:

`filteredRatings = userGroups.filter(lambda x: x.Rating.size >= 30)`

You can do this (they are functionally equivalent!):
```
def filterFewestRatings(dataset):
    return dataset.Rating.size >= 30
    
filteredRatings = userGroups.filter(filterFewestRatings)
```

Here, we've replaced the variable `x` in the anonymous function with a more easily understood variable `dataset` in the named (non-anonymous) function `filterFewestRatings`. This has the same exact functionality as the **anonymous function**, but is clearly more code to write. 

**It's your choice as to how you want to implement small functions like this.**

Lets look at filtered ratings and see how many ratings and users we eliminated before we recompute our answers:

In [82]:
print("Original number of ratings:",ratingData.shape[0])
print("Filtered number of ratings:",filteredRatings.shape[0])
print("Fraction of ratings eliminated:",(1.0-float(filteredRatings.shape[0])/ratingData.shape[0]))

Original number of ratings: 1000209
Filtered number of ratings: 982040
Fraction of ratings eliminated: 0.0181652034725


In [83]:
print("Original number of users:",ratingData.UserID.unique().size)
print("Filtered number of users:",filteredRatings.UserID.unique().size)
print("Fraction of users eliminated:",(1.0-float(filteredRatings.UserID.unique().size)/ratingData.UserID.unique().size))

Original number of users: 6040
Filtered number of users: 5289
Fraction of users eliminated: 0.124337748344


So we got rid of about 2% of the ratings, but over 12% of the users!

Now that we've filtered, you should be able to do the rest and recompute the answers:

In [85]:
filteredMonthGroups = filteredRatings.groupby("month")
filteredMonthAggs = filteredMonthGroups.Rating.agg([np.mean,np.median,np.std,np.max]).reset_index()

In [86]:
filteredMonthAggs

,month,mean,median,std,amax
0,1,3.540490,4,1.074986,5
1,2,3.521995,4,1.089586,5
2,3,3.452294,4,1.079984,5
3,4,3.518530,4,1.096383,5
4,5,3.600756,4,1.137382,5
5,6,3.614453,4,1.109368,5
6,7,3.614583,4,1.091591,5
7,8,3.564065,4,1.122199,5
8,9,3.600072,4,1.112411,5
9,10,3.606638,4,1.086538,5


To hammer all of this home, try to answer the following:

**What were the ID(s) of the average highest-rated movie in the first half of the year for those movies that were rated at least 5 times within that timespan?**

Fill in the next cell by replacing the `pass` (or make more cells if you need to) and tell me when you know the answer (but quietly, so you don't tell everyone else!).

In [90]:
ratingData['MovieID']
# get back to this one later

0          1193
1           661
2           914
3          3408
4          2355
5          1197
6          1287
7          2804
8           594
9           919
10          595
11          938
12         2398
13         2918
14         1035
15         2791
16         2687
17         2018
18         3105
19         2797
20         2321
21          720
22         1270
23          527
24         2340
25           48
26         1097
27         1721
28         1545
29          745
           ... 
1000179    2762
1000180    1036
1000181     508
1000182    1041
1000183    3735
1000184    2791
1000185    2794
1000186     527
1000187    2003
1000188     535
1000189    2010
1000190    2011
1000191    3751
1000192    2019
1000193     541
1000194    1077
1000195    1079
1000196     549
1000197    2020
1000198    2021
1000199    2022
1000200    2028
1000201    1080
1000202    1089
1000203    1090
1000204    1091
1000205    1094
1000206     562
1000207    1096
1000208    1097
Name: MovieID, dtype: in

A couple more data analysis functions before I set you loose on a completely new dataset!

**pandas** provides some pretty cool basic statistical functionality apart from the really simple functions we've used so far (like `mean`, `std`, `max`, and `size`).

What I'm talking about are statistical functions that compare two variables, specifically  **covariance** and **correlation**. 

Briefly, **covariance** is a way to tell whether two variables tend to move in the same or opposite directions, but cannot measure the absolute strength of this relationship (if two variables show positive covariance, it means that as one increases, so does the other one, and vice versa). 

**Correlation** on the other hand, can tell you both whether two variables tend to move with or against each other, and how strong this relationship between them is.

I will be showing you a fairly contrived example here, but you will be using this functionality in the actual assignment you'll be working on.

I will keep this short and sweet, because I know you are dying to get started on your own!

To correlate two columns using the standard pearson correlation in **pandas** simply do something like this, where we correlate `month` with `Rating`:

In [120]:
ratingData.month.corr(ratingData.Rating,method="spearman")

-0.0010186797292071324

If you want to get the full correlation matrix among all variables that are of a numeric type, just call `corr` on the whole `DataFrame`. **pandas** will know to only make the correlations among numeric columns only and will exclude all non-numeric columns from the resulting correlation matrix:

In [121]:
ratingData.corr()

,UserID,MovieID,Rating,year,month
UserID,1.000000,-0.017739,0.012303,-0.094288,-0.633559
MovieID,-0.017739,1.000000,-0.064042,0.039194,-0.003123
Rating,0.012303,-0.064042,1.000000,-0.024523,0.000979
year,-0.094288,0.039194,-0.024523,1.000000,-0.432945
month,-0.633559,-0.003123,0.000979,-0.432945,1.000000


Obviously, this is a contrived example because **the only truly numeric column here is Rating** all the other columns (except for `FormattedTimestamp`) are numeric labels for categorical items. 

What this means is that there is no way we can actually say that one `UserID` is greater than some other `UserID` because the mapping from the actual user's identity to the `UserID` number associated with them is arbitrary, and the same thing applies to every other numeric column here except for `Rating`. 

And because we can't rank any of the entries in the other columns in any meaningful way, **the correlation between them is meaningless.** 

I simply wanted to show you how you would do this, given the right situation (which you will experience shortly!).

To consolidate what you've learned answer the following questions about this dataset:

1. Which user rated the most movies?
* Which movie id got the highest average rating?
* Which movie id had the most varied rating?
  * Of those movies that have been rated at least 5 times?
* In which month were raters the most generous on average?
* Which month had the most variation in ratings?
* Which user had the most variation in ratings?
* On what day of the week did users rate the most movies?
* Which hour in the day did users rate the most movies?
* What hour in the day had the highest average movie rating?
* Which user gave movies the highest average rating?

In [171]:
# 1. Which user rated the most movies
ratingData_UserID=ratingData.UserID.value_counts()[:1]
ratingData_UserID
# ratingData.UserID.unique()
# ratingData.UserID.value_counts()
# The user who rated the most movies was 4169. They rated 2314 movies.

4169    2314
Name: UserID, dtype: int64

In [199]:
# 2. Which movie ID got the highest average rating?
movieIDGroups = ratingData.groupby("MovieID")
print(movieIDGroups.describe())

                    Rating       UserID          day        month         year
MovieID                                                                       
1       count  2077.000000  2077.000000  2077.000000  2077.000000  2077.000000
        mean      4.146846  3053.819933    15.490130     8.824747  2000.063072
        std       0.852349  1750.606746     8.871949     2.568916     0.285097
        min       1.000000     1.000000     1.000000     1.000000  2000.000000
        25%       4.000000  1532.000000     7.000000     7.000000  2000.000000
        50%       4.000000  3146.000000    17.000000     9.000000  2000.000000
        75%       5.000000  4571.000000    22.000000    11.000000  2000.000000
        max       5.000000  6040.000000    31.000000    12.000000  2003.000000
2       count   701.000000   701.000000   701.000000   701.000000   701.000000
        mean      3.201141  3027.977175    15.758916     8.718973  2000.108417
        std       0.983172  1749.000320     8.858915

In [210]:
ratingsMean = movieIDGroups.mean()
ratingsMean['Rating']

MovieID
1       4.146846
2       3.201141
3       3.016736
4       2.729412
5       3.006757
6       3.878723
7       3.410480
8       3.014706
9       2.656863
10      3.540541
11      3.793804
12      2.362500
13      3.262626
14      3.542484
15      2.458904
16      3.793255
17      4.027545
18      3.337580
19      2.480720
20      2.537500
21      3.623894
22      3.349206
23      2.857143
24      3.179487
25      3.651020
26      3.530000
27      2.934426
28      4.055866
29      4.062035
30      3.648649
          ...   
3923    2.576923
3924    2.923077
3925    3.854839
3926    3.361111
3927    3.580460
3928    3.441748
3929    3.993197
3930    3.255605
3931    2.037037
3932    3.750000
3933    2.250000
3934    3.035714
3935    3.000000
3936    3.723214
3937    2.940741
3938    1.925926
3939    1.961538
3940    1.733333
3941    2.045455
3942    1.785714
3943    3.052083
3944    2.111111
3945    1.488372
3946    2.260000
3947    3.472727
3948    3.635731
3949    4.115132
3950  

In [213]:
ratingsMeanMax = ratingsMean[ratingsMean==ratingsMean.max()]
ratingsMeanMax

,UserID,Rating,year,month,day
MovieID,,,,,
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [198]:
# print(ratingGroups.mean())
ratingGroups[['Rating','MovieID']].max().mean()
ratingGroups.Rating.max().mean()

4.796006475984889